# Settings

In [ ]:
!git clone https://github.com/QuocThinh73/HCMAI2025.git

In [ ]:
%cd HCMAI2025

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install faiss-cpu==1.11.0

# Tải và Giải nén

In [ ]:
!gdown --id 1qROoBnsGqt3hoNOz9AwA1cOO207E12Gd

In [ ]:
!unzip data.zip -d .

# Build FAISS

In [ ]:
import glob

image_paths = sorted(glob.glob("data/keyframes/*.jpg", recursive=True))
image_paths

In [8]:
from models.clip import CLIP

model = CLIP()

100%|███████████████████████████████████████| 338M/338M [00:24<00:00, 14.7MiB/s]


In [9]:
from faiss_index import FaissIndex

my_faiss = FaissIndex(model=model)

In [10]:
model_name = "clip"

my_faiss.build(image_paths=image_paths, model_name=model_name)

# Test

In [11]:
import matplotlib.pyplot as plt
from PIL import Image

def display_results(paths, scores=None, max_images=5):
    """Display image results in Colab."""
    n = min(len(paths), max_images)
    fig, axes = plt.subplots(1, n, figsize=(15, 3))

    if n == 1:
        axes = [axes]

    for i in range(n):
        try:
            img = Image.open(paths[i])
            axes[i].imshow(img)
            if scores is not None:
                axes[i].set_title(f"Score: {scores[i]:.4f}")
            axes[i].axis('off')
        except Exception as e:
            print(f"Error displaying image {paths[i]}: {e}")

    plt.tight_layout()
    plt.show()

In [12]:
# Load FAISS index
index_path = f"database/{model_name}_faiss.bin"
id2path_path = f"database/{model_name}_id2path.pkl"

my_faiss.load(index_path, id2path_path)

In [ ]:
query = "Docter"
scores, indices, paths = my_faiss.text_search(query, top_k=5)

for i, (score, idx, path) in enumerate(zip(scores, indices, paths)):
           print(f"{i+1}. Score: {score:.4f}, Index: {idx}, Path: {path}")

display_results(paths, scores, 5)